# GSM interface model with scalars and vector variables

Based on the notebook 01 here we are separating the general part of the
derivation valid for any kind of generalized standard model and the particular
formulation of the external and internal state representation and the free energy and dissipation potentials

In [ ]:
%matplotlib widget
from bmcs_matmod.gsm.gsm_symb import GSMSymb as GSM
import matplotlib.pylab as plt 
import sympy as sp
from bmcs_utils.api import Cymbol
import numpy as np
sp.init_printing()

In [ ]:
gsm = GSM.load_from_disk('gsm_3DNT_TEVPD_LIKH_double_cap')

In [ ]:
_u_a = np.array([-20, 3, 0])
_T = np.array([20])
material_params = dict(
    E_T_=1, gamma_T_=1, K_T_=1, S_T_=1000, c_T_=1, f_s_=1, 
    E_N_=1, gamma_N_=0, K_N_=0, S_N_=1000, c_N_=1, m_=0.1, f_t_=1, f_c_=20, f_c0_=10, 
    r_N_=2, r_T_=2, zeta_=0.0001, eta_N_=1, eta_T_=1, d_N_=0, alpha_therm_=1e-5, 
    T_0_=20, C_v_=1, beta_=1
)
_Eps_B00 = np.zeros((gsm.n_Eps_explicit,), np.float_ )
_f = gsm.get_f(_u_a, 20, _Eps_B00, _Eps_B00, **material_params)
_df_dlambda = gsm.get_df_dlambda(_u_a, 20, _Eps_B00, _Eps_B00, **material_params)
_f2, _df_dlambda2, _Sig_B00 = gsm.get_f_df_Sig(_u_a, _T, _Eps_B00, _Eps_B00, **material_params)
_Phi = gsm.get_Phi(_u_a, 20, _Eps_B00, _Sig_B00, **material_params)
_f2, _df_dlambda2, _Sig_B00, _Phi

## Threshold function and rate of dissipation potential

In [ ]:
n_u = 80
n_u_c = complex(0, n_u)
n_t = 30
t_t = np.linspace(0, 1, n_t)
u_N_range, u_T_range = np.mgrid[-20:2:n_u_c, -2:2:n_u_c]
_u_aIJ = np.zeros((3,) + u_N_range.shape, np.float_)
_u_aIJ[0] = u_N_range
_u_aIJ[1] = u_T_range
_u_tIJa = np.einsum('t,aIJ->tIJa', t_t, _u_aIJ)
_T_tIJ = np.ones((n_t, n_u, n_u), dtype=np.float_) * 20

In [ ]:
# u_N_range, u_T_range = np.mgrid[-22:3:500j, -5:5:500j]
# u_N_range, u_T_range = np.mgrid[-5:5:500j, -5:5:500j]
#_u_IJ = np.zeros((3,) + u_N_range.shape, np.float_)
_u_aIJ = np.moveaxis(_u_tIJa[-1], -1, 0)
_T_IJ = _T_tIJ[0]
# _u_IJ[0] = u_N_range
# _u_IJ[2] = u_T_range
_Eps_BIJ = np.zeros((gsm.n_Eps_explicit, *u_N_range.shape), np.float_)
_T_IJ = np.ones_like(u_N_range) * 20
_f_IJ, _df_dlambda_IJ, _Sig_IJ = gsm.get_f_df_Sig(_u_aIJ, _T_IJ, _Eps_BIJ, _Eps_BIJ, **material_params)
fig, ax = plt.subplots(1, 1, figsize=(8, 4), tight_layout=True)
fig.canvas.header_visible = False
ax.contour(u_N_range, u_T_range, _f_IJ, levels=[0]);
ax.contourf(u_N_range, u_T_range, _df_dlambda_IJ);
ax.axis('equal');

## Quantitative verification for a single time step, single point

In [ ]:
gsm.vp_on = True
gsm.update_at_k = False
material_params = dict(
    E_T_=1, gamma_T_=0, K_T_=1, S_T_=0.5e+0, c_T_=1, f_s_=1, 
    E_N_=1, gamma_N_=0, K_N_=1, S_N_=0.5e+0, c_N_=1, m_=0.05, f_t_=1, f_c_=20, f_c0_=10, 
    zeta_=0, eta_N_=0.005, eta_T_=0.005, d_N_=0, alpha_therm_=1e-5, 
    T_0_=20, C_v_=1, beta_=1
)
response = gsm.get_response(_u_tIJa, _T_tIJ, t_t, **material_params)

### Plot results within the displacement domain

In [ ]:
u_tIJa, T_tIJ, Eps_tIJb, Sig_tIJb = response[1], response[2], response[3], response[4] # displacement and state history
u_atIJ, Eps_btIJ, Sig_btIJ = [np.moveaxis(rvar, -1, 0) for rvar in (u_tIJa, Eps_tIJb, Sig_tIJb)]
u_N_tIJ, u_T_tIJ = u_atIJ[[0,1]]
#
Eps_BtIJ = gsm.Eps_as_blocks(Eps_btIJ)
_u_p_atIJ = Eps_BtIJ[0]
Sig_BtIJ = gsm.Eps_as_blocks(Sig_btIJ)
_sig_atIJ = Sig_BtIJ[0]
_u_e_aIJ = _u_aIJ - _u_p_atIJ[:,-1]
_u_e_N_IJ, _u_e_T_IJ = _u_e_aIJ[[0,1]]
# last converged step
u_N_IJ, u_T_IJ = u_N_tIJ[-1], u_T_tIJ[-1]
omega_N_tIJ, omega_T_tIJ = Eps_BtIJ[-1] # damage block
sig_N_tIJ, sig_Tx_tIJ, sig_Ty_tIJ = _sig_atIJ # stress block 
# plot the original threshold function
_Eps_bIJ = np.zeros((gsm.n_Eps_explicit,) + u_N_range.shape, np.float_)
_f_IJ, _df_dlambda_IJ, _ = gsm.get_f_df_Sig(_u_aIJ, _T_IJ, _Eps_bIJ, _Eps_bIJ, **material_params)
# Plot area
fig, ((ax_u_p, ax_sig_N, ax_sig_T), (ax_T, ax_omega_N, ax_omega_T)) = plt.subplots(
    2, 3, figsize=(10, 6), tight_layout=True
    )
fig.canvas.header_visible = False
# displacement
_v_N_range = np.array([u_N_IJ.flatten(), _u_e_N_IJ.flatten()]) 
_v_T_range = np.array([u_T_IJ.flatten(), _u_e_T_IJ.flatten()]) 
ax_u_p.plot(_v_N_range, _v_T_range, marker='o', markersize=1, color='black', lw=0.2)
ax_u_p.axis('equal');
ax_u_p.set_title(r'$u_\mathrm{e}$')
ax_u_p.set_xlabel(r'$u_\mathrm{N}$')
ax_u_p.set_ylabel(r'$u_\mathrm{T}$')
# sigma _N
ax_sig_N.contourf(u_N_tIJ[-1], u_T_tIJ[-1], sig_N_tIJ[-1], cmap='coolwarm');
ax_sig_N.contour(u_N_tIJ[-1], u_T_tIJ[-1], _f_IJ, levels=[0], colors=['black']);
ax_sig_N.axis('equal')
ax_sig_N.set_title(r'$\sigma\mathrm{N}$')
ax_sig_N.set_xlabel(r'$u_\mathrm{N}$')
ax_sig_N.set_ylabel(r'$u_\mathrm{T}$')
# sigma _T
ax_sig_T.contourf(u_N_tIJ[-1], u_T_tIJ[-1], sig_Tx_tIJ[-1], cmap='coolwarm');
ax_sig_T.contour(u_N_tIJ[-1], u_T_tIJ[-1], _f_IJ, levels=[0], colors=['black']);
ax_sig_T.axis('equal')
ax_sig_T.set_title(r'$\sigma\mathrm{T}$')
ax_sig_T.set_xlabel(r'$u_\mathrm{N}$')
ax_sig_T.set_ylabel(r'$u_\mathrm{T}$')
# 
max_omega_N = np.max(omega_N_tIJ[-1])
ax_omega_N.contourf(u_N_tIJ[-1], u_T_tIJ[-1], omega_N_tIJ[-1], levels=np.linspace(1e-10, max_omega_N, 10), cmap='RdPu');
ax_omega_N.contour(u_N_tIJ[-1], u_T_tIJ[-1], _f_IJ, levels=[0], colors=['black']);
ax_omega_N.axis('equal')
ax_omega_N.set_title(r'$\omega_\mathrm{N}$')
ax_omega_N.set_xlabel(r'$u_\mathrm{N}$')
ax_omega_N.set_ylabel(r'$u_\mathrm{T}$')
max_omega_T = np.max(omega_T_tIJ[-1])
ax_omega_T.contourf(u_N_tIJ[-1], u_T_tIJ[-1], omega_T_tIJ[-1], levels=np.linspace(1e-10, max_omega_T, 10), cmap='RdPu');
ax_omega_T.contour(u_N_tIJ[-1], u_T_tIJ[-1], _f_IJ, levels=[0], colors=['black']);
ax_omega_T.axis('equal')
ax_omega_T.set_title(r'$\omega_\mathrm{T}$')
ax_omega_T.set_xlabel(r'$u_\mathrm{N}$')
ax_omega_T.set_ylabel(r'$u_\mathrm{T}$')
ax_T.contourf(u_N_tIJ[-1], u_T_tIJ[-1], T_tIJ[-1], cmap='coolwarm')
ax_T.contour(u_N_tIJ[-1], u_T_tIJ[-1], _f_IJ, levels=[0], colors=['black']);
ax_T.axis('equal')
ax_T.set_title(r'$T$')
ax_T.set_xlabel(r'$u_\mathrm{N}$')
ax_T.set_ylabel(r'$u_\mathrm{T}$')
max_omega_N, max_omega_T, np.max(T_tIJ)